# dPULearn: Learning from unbalanced data

In [ ]:
import aaanalysis as aa
aa.options["verbose"] = False

More details on the ``dPULearn`` and ``dPULearn`` classes are given in the [PU Learning API](https://aaanalysis.readthedocs.io/en/latest/api.html#pu-learning). More background in PU learning are available in our [dPULearn Usage Principles](https://aaanalysis.readthedocs.io/en/latest/index/usage_principles/pu_learning.html) section.